In [2]:
import pandas as pd
import numpy as np
from scipy import sparse
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import FreqDist
from sklearn.svm import SVC
from sklearn.metrics import precision_score, accuracy_score, recall_score
from sklearn.model_selection import train_test_split
import nltk
from nltk import pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('stopwords')
#https://github.com/duyet/demo-text-classification/blob/master/classification-with-tfidf-svm.ipynb
#https://github.com/sid-thiru/Text-Classification-with-TFIDF-and-sklearn/blob/master/sklearn_classifiers.py
#https://github.com/jonathanoheix/Sentiment-analysis-with-hotel-reviews/blob/master/Sentiment%20analysis%20with%20hotel%20reviews.ipynb

emoji = [':)', '(;', '(:', ';)', '|:', ':|', '@_@', '):',':(', '),:', ':,(', ')\';', ':\'(',':D', ':}', '{:', '}:', ':{',
         ':]', ':[', '[:', ']:', '\\:', '/:', ':/', ':\\', ':p', ';p', '*_*',
         ':*', '*:', '^^','^_^', '^-^', ':O', 'O:', ':o', 'o:']


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:

def read_train_files(path):
    train_data = pd.read_csv(path + "/test_data.csv")
    train_results = pd.read_csv(path + "/train_results.csv")
    return train_data, train_results


def get_words(string):
    string = str(string)
    string = re.sub('[\n\t]', ' ', string)
    for e in emoji:
        if e in string:
            string = string.replace(e, " {} ".format(e))

    string_list = string.split(" ")

    new_list = []
    for word in string_list:
        if word == '':
            continue
        elif word[:5] == "http:":
            # new_list.append(word)
            continue
        elif word[0] == "@":
            continue
        elif word in emoji:
            new_list.append(word)
        else:
            split_word = re.sub('[\']+', '', word) #remove '
            split_word = re.sub('[^A-Za-z\']+', ' ', split_word)
            split_word = re.sub(r'(.)\1+', r'\1\1', split_word.lower()) #remove repeated letters

            split_word = split_word.split(" ")
            new_list += split_word


    new_list = [i for i in new_list if i != '']
    new_list = normalize(new_list)


    return new_list


In [4]:

def get_wordnet_pos(pos_tag):
    if pos_tag.startswith('J'):
        return wordnet.ADJ
    elif pos_tag.startswith('V'):
        return wordnet.VERB
    elif pos_tag.startswith('N'):
        return wordnet.NOUN
    elif pos_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN



In [5]:

def add_to_dictionary(text_list, all_words_dict):
    for word in text_list:
        if word in all_words_dict:
            all_words_dict[word] += 1
        else:
            all_words_dict[word] = 1
    return all_words_dict

def normalize(words): #input: lists of words in each sentence
    """
    Removes stop words and lowercases the text.
    Returns a list of words
    """
    stop_words = stopwords.words('english')
    text = [w.lower() for w in words if (w.lower() not in stop_words)]
    # ps = PorterStemmer()
    # lemmatize text

    pos_tags = pos_tag(text)
    text = [WordNetLemmatizer().lemmatize(t[0], get_wordnet_pos(t[1])) for t in pos_tags]
    return text


In [6]:

def count_all_words():
    with open("processed strings_lem_test.json", 'r') as file:
        train_data = json.load(file)
    all_words_dict = {}
    for i in range(len(train_data)):
        # print(i)
        # row_text = train_data.iloc[i]['text']
        row_text = train_data[i]
        text_list = row_text.split(" ")
        all_words_dict = add_to_dictionary(text_list, all_words_dict)
    # print(all_words_dict)
    all_words_dict_sorted = dict(sorted(all_words_dict.items(), key=lambda item: item[1], reverse=True))
    with open("processed_words_lem_dict_sorted_test.json", "w") as write_file:
        json.dump(all_words_dict_sorted, write_file, indent=1)

In [7]:

def remove_few_words(words_number_limit):
    with open("processed_words_lem_dict_sorted.json", 'r') as file: #we should still process test file with this dict
        dictt = json.load(file)
    filter_dict = dict((key, value) for (key, value) in dictt.items() if value > words_number_limit)
    print(len(filter_dict))

    with open("processed strings_lem_test.json", 'r') as file:
        strings = json.load(file)
    new_strings_list = []
    for row in strings:
        s = row.split(" ")
        new_string = [word for word in s if word in filter_dict and len(word)> 1]
        new_string = " ".join(new_string)
        new_strings_list += [new_string]
    with open("processed strings_remove_few_words_lem100_test.json", 'w') as file:
        print("here")
        json.dump(new_strings_list ,file)
    return new_strings_list


In [8]:
def process_all_words(path):
    train_data, train_results = read_train_files(path)
    # train_data = train_data.iloc[:10]
    all_words_list = []
    for i in range(len(train_data)):
        print(i)
        row_text = train_data.iloc[i]['text']
        text_list = get_words(row_text)
        all_words_list.append(" ".join(text_list))

    # save the list
    with open("processed strings_lem_test.json", "w") as write_file:
        json.dump(all_words_list, write_file)
    return all_words_list

In [9]:

def tfidf(data):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_data = tfidf_vectorizer.fit_transform(data).astype("float16")
    words = tfidf_vectorizer.get_feature_names()
    with open("tfidf_words_lem_removed_words100_test.json", "w") as write_file:
        json.dump(words, write_file)

    sparse.save_npz("tfidf_matrix_lem_removed_words100_test.npz", tfidf_data)
    print(len(words))
    return tfidf_data


def BoW(data, maxx = 0.6, minn = 0.0001):
    bow_vectorizer = CountVectorizer(max_df = maxx, min_df = minn)
    bow_data = bow_vectorizer.fit_transform(data)
    words = bow_vectorizer.get_feature_names()
    with open("bow_words_lem_removed_words100_test.json", "w") as write_file:
        json.dump(words, write_file)

    sparse.save_npz("bow_matrix_lem_removed_words100_test.npz", bow_data)
    return bow_data


In [10]:

def test_SVM(x_train, x_test, y_train, y_test, x_real_test):
    SVM = SVC(kernel = 'rbf', verbose=True, C = 50, gamma = 0.0045)
    #35 , 0.1
    #40, 0.09
    # C = 50, gamma = 0.002  train 9000 72.95% remove 500
    #C = 50, gamma = 0.003  train 9000 72.97% remove 500
    #C = 50, gamma = 0.003  train 19000 74.07% remove 500
    #C = 50, gamma = 0.003  train 39000 75.14% remove 200
    #C = 50, gamma = 0.003  train 39000 75.26% remove 100
    #C = 50, gamma = 0.003  train 79000 75.97% remove 100
    #C = 50, gamma = 0.004  train 79000 75.98% remove 100
    #C = 100, gamma = 0.004  train 79000 75.82% remove 100
    ###lem
    #C = 50, gamma = 0.004  train 9000 73.37% remove 100
    #C = 50, gamma = 0.0045  train 9000 73.38% remove 100
    SVMClassifier = SVM.fit(x_train, y_train)
    print(y_train)
    print("done training")
    predictions = SVMClassifier.predict(x_test)
    a = accuracy_score(y_test, predictions)
    p = precision_score(y_test, predictions, average = 'weighted')
    r = recall_score(y_test, predictions, average = 'weighted')
    print(a, r, p)
    predictions = SVMClassifier.predict(x_real_test)
    with open("prediction_svm_linear100.json", "w") as write_file:
        json.dump([int(res) for res in predictions], write_file)
    return a, p, r


In [11]:
! git clone https://github.com/Xtra-Computing/thundersvm.git
! cd thundersvm && mkdir build && cd build && cmake .. && make -j
! python /content/thundersvm/python/setup.py install

fatal: destination path 'thundersvm' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
running install
running bdist_egg
running egg_info
writing thundersvm.egg-info/PKG-INFO
writing dependency_links to thundersvm.egg-info/dependency_links.txt
writing requirements to thundersvm.egg-info/requires.txt
writing top-level names to thundersvm.egg-info/top_level.txt
Generating grammar tables from /usr/lib/python3.8/lib2to3/Grammar.txt
Generating grammar tables from /usr/lib/python3.8/lib2to3/PatternGrammar.txt
package init file 'thundersvm/__init__.py' not found (or not a regular file)
reading manifest file 'thundersvm.egg-info/SOURCES.txt'
writing manifest file 'thundersvm.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
copying thundersvm.egg-info/PKG-INFO -> build/bdist.linux-x86_64/egg/EGG-IN

In [12]:

!sudo apt-get install cuda-9.0
import importlib
!pip install importlib
from importlib.machinery import SourceFileLoader
thundersvm = SourceFileLoader("thundersvm", "/content/thundersvm/python/thundersvm/thundersvm.py").load_module()



Reading package lists... Done
Building dependency tree       
Reading state information... Done
Note, selecting 'libcuda-9.0-1' for regex 'cuda-9.0'
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!nvcc --version
!pip install thundersvm

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
path  = "/content/drive/MyDrive/Kaggle 2 data/"
# data = process_all_words(path)
# count_all_words()
# remove_few_words(100)
# with open("processed strings_remove_few_words_lem100_test.json", 'r') as file:
#     data = json.load(file)
# tfidf(data)
# BoW(data)

In [16]:
from thundersvm import SVC

training = sparse.load_npz("tfidf_matrix_lem_removed_words100.npz")
x_real_test = sparse.load_npz("tfidf_matrix_lem_removed_words100_test.npz")

train_results = pd.read_csv("/content/drive/MyDrive/Kaggle 2 data/train_results.csv")
train_results = list(train_results['target'])
# #

new_data_test = []
for i in range(len(train_results)):
    if train_results[i] == 'neutral':
        res = 1
    elif train_results[i] == 'negative':
        res = 0
    elif train_results[i] == 'positive':
        res = 2
    new_data_test.append(int(res))

x_train = training[:475000]
x_test = training[475000:]
y_train = new_data_test[:475000]
y_test = new_data_test[475000:]

#acc : 75.89 -> 90000 train, C = 0.6, gamma = 0.1, remove_words(200) : 3307 features
#acc : 75.43 -> 90000 train, C = 0.6, gamma = 0.1, remove_words(500): 1776 features
print(x_train.shape)
print(x_test.shape)
# # #
# # # #test a classifier
accuracy, precision, recall = test_SVM(x_train, x_test, y_train, y_test, x_real_test)
# print(accuracy, precision, recall)

(475000, 5324)
(565323, 5324)
[2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 2, 0, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 0, 0, 2, 2, 0, 0, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 0, 0, 2, 2, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 2, 0, 0, 2, 0, 0, 2, 0, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 2, 2, 0, 2, 2, 0, 0, 0, 0, 2, 0, 2, 0, 2, 2, 0, 2, 

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.772294776614431 0.772294776614431 0.7732439345660596


In [18]:
print(accuracy)
with open("prediction_svm_linear100.json", 'r') as file:
  data = json.load(file)
df = pd.DataFrame(data, columns =['target'])
df.reset_index(inplace=True)
df = df.rename(columns={'index': 'id'})
df.to_csv("prediction_linear_svm_rbf100_c50g0045lem-acc7722.csv", index=False)

0.772294776614431
